In [1]:
!pip install joblib

In [2]:

%matplotlib inline
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
from numpy import loadtxt
import os
import scipy
import matplotlib.pyplot as plt
from matplotlib import rcParams
import warnings
import sklearn
from sklearn import preprocessing
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
import seaborn as sb

from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
#from sklearn.linear_model import LogisticRegressionCV
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import svm
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, VotingClassifier
import xgboost as xgb

from xgboost import XGBClassifier
#from sklearn.model_selection import train_test_split
#from sklearn import joblib
import joblib
from joblib import dump, load
import pickle


In [3]:
#To view all columns in DF ,expanded without ellipses
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', -1)

In [4]:
df=pd.read_csv("NFL_datasetForML.csv")
df_2=df.drop(['Unnamed: 0','boxscore_id', 'roof', 'surface', 'temp', 'humidity', 'wind_chill', 'wind_mph', 'Won'], axis=1)
df_2.head(5)

,season,week,team,home,pass_attempts,pass_completions,pass_yards,net_pass_yards,pass_tds,rush_attempts,rush_yards,rush_tds,total_yards,first_downs,sacks,sacks_yards,pass_interceptions,fumbles,fumbles_lost,turnovers,time_of_possession,pentalties,penalty_yards,third_down_attempts,third_down_conversions,fourth_down_attempts,fourth_down_conversions,Score
0,2002,1,sfo,0,26,16,166.0,166,1.0,25,113.0,0.0,279,13,0,0,1.0,0,0,1,27:32:00,0,0,4,12,0,0,16
1,2002,1,nyg,1,45,28,342.0,318,0.0,22,43.0,1.0,361,21,3,24,3.0,0,0,3,32:28:00,3,24,9,16,0,1,13
2,2002,1,det,0,36,17,220.0,206,1.0,19,51.0,2.0,257,15,3,14,1.0,2,1,2,25:36:00,3,14,7,14,0,1,21
3,2002,1,mia,1,27,18,207.0,207,3.0,41,182.0,3.0,389,27,0,0,0.0,2,0,0,34:24:00,0,0,6,12,1,1,49
4,2002,1,clt,0,31,19,211.0,203,3.0,28,104.0,0.0,307,18,1,8,0.0,2,2,2,27:27:00,1,8,9,14,0,0,28


In [5]:
type(df_2)
df_2=pd.DataFrame(df_2)
type(df_2)
df_2.shape

(9078, 28)

In [6]:
# from sklearn import preprocessing
# le = preprocessing.LabelEncoder()
# X=df_2

# for i in range(0,X.shape[1]):
#     if X.dtypes[i]=='object':
#         X[X.columns[i]] = le.fit_transform(X[X.columns[i]],axis=0)

In [7]:
# X.head(1)

In [8]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
df_2['team']=df_2['team'].tolist()
df_2['team'] = le.fit_transform(df_2['team'])

df_2['team']


0       29
1       19
2       10
3       15
4       6 
5       13
6       16
7       3 
8       26
9       2 
10      14
11      5 
12      0 
13      11
14      20
15      1 
16      27
17      4 
18      7 
19      31
20      22
21      21
22      17
23      30
24      28
25      24
26      25
27      9 
28      8 
29      12
        ..
9048    24
9049    14
9050    5 
9051    26
9052    27
9053    9 
9054    6 
9055    21
9056    6 
9057    12
9058    28
9059    8 
9060    27
9061    26
9062    22
9063    3 
9064    6 
9065    14
9066    8 
9067    25
9068    27
9069    18
9070    22
9071    17
9072    25
9073    17
9074    18
9075    14
9076    25
9077    18
Name: team, Length: 9078, dtype: int32

In [9]:
df_2['time_of_possession'] = le.fit_transform(df_2['time_of_possession'])
df_2['time_of_possession']

0       495 
1       791 
2       379 
3       907 
4       490 
5       796 
6       734 
7       552 
8       412 
9       874 
10      659 
11      627 
12      698 
13      1164
14      119 
15      1181
16      1108
17      178 
18      379 
19      907 
20      595 
21      691 
22      1247
23      732 
24      352 
25      934 
26      749 
27      537 
28      408 
29      878 
       ...  
9048    879 
9049    407 
9050    140 
9051    1146
9052    535 
9053    751 
9054    1230
9055    68  
9056    717 
9057    569 
9058    353 
9059    933 
9060    863 
9061    423 
9062    691 
9063    595 
9064    82  
9065    1213
9066    270 
9067    1016
9068    146 
9069    1140
9070    176 
9071    1110
9072    799 
9073    684 
9074    1308
9075    111 
9076    453 
9077    833 
Name: time_of_possession, Length: 9078, dtype: int32

In [10]:
def get_matRx_4_gvn_Year(year):
    
    if(year>=2002):
        
        groupby_Season = df_2.groupby('season')
        df_for_Year_Passed_1 = groupby_Season.get_group(year-1)
        groupby_Team_Avg_Values_1 = df_for_Year_Passed_1.groupby('team', as_index=False).mean()
        groupby_Team_Avg_Values_1 = groupby_Team_Avg_Values_1[['season', 'week','team', 'home', 'pass_attempts', 'pass_completions', 'pass_yards', 'net_pass_yards', 'pass_tds', 'rush_attempts', 'rush_yards', 'rush_tds', 'total_yards', 'first_downs', 'sacks', 'sacks_yards', 'pass_interceptions', 'fumbles', 'fumbles_lost', 'turnovers', 'time_of_possession', 'pentalties', 'penalty_yards', 'third_down_attempts', 'third_down_conversions', 'fourth_down_attempts', 'fourth_down_conversions','Score']]
        
        df_for_Year_Passed_2 = groupby_Season.get_group(year-2)
        groupby_Team_Avg_Values_2 = df_for_Year_Passed_2.groupby('team', as_index=False).mean()
        groupby_Team_Avg_Values_2 = groupby_Team_Avg_Values_2[['season', 'week','team', 'home', 'pass_attempts', 'pass_completions', 'pass_yards', 'net_pass_yards', 'pass_tds', 'rush_attempts', 'rush_yards', 'rush_tds', 'total_yards', 'first_downs', 'sacks', 'sacks_yards', 'pass_interceptions', 'fumbles', 'fumbles_lost', 'turnovers', 'time_of_possession', 'pentalties', 'penalty_yards', 'third_down_attempts', 'third_down_conversions', 'fourth_down_attempts', 'fourth_down_conversions','Score']]
    
    result = pd.concat([groupby_Team_Avg_Values_1, groupby_Team_Avg_Values_2], axis=0).dropna(axis=1).groupby('team', as_index=False).mean()
    result = result[['season', 'week','team', 'home', 'pass_attempts', 'pass_completions', 'pass_yards', 'net_pass_yards', 'pass_tds', 'rush_attempts', 'rush_yards', 'rush_tds', 'total_yards', 'first_downs', 'sacks', 'sacks_yards', 'pass_interceptions', 'fumbles', 'fumbles_lost', 'turnovers', 'time_of_possession', 'pentalties', 'penalty_yards', 'third_down_attempts', 'third_down_conversions', 'fourth_down_attempts', 'fourth_down_conversions','Score']]
    scaled = preprocessing.MinMaxScaler()
    scaled_result = scaled.fit_transform(result)
    return scaled_result   



In [11]:
get_matRx_4_gvn_Year1=get_matRx_4_gvn_Year(2012)
#predictor_df=get_matRx_4_gvn_Year.values

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


In [12]:
type(get_matRx_4_gvn_Year1)

numpy.ndarray

In [13]:
get_matRx_4_gvn_Year1.shape


(32, 28)

In [14]:
# Team_list = list(le.inverse_transform(df_2['team']))
# Team_list

In [15]:
# le_name_mapping = dict(zip(le.classes_, le.inverse_transform(le.classes_)))
# print(le_name_mapping)

In [16]:
logit_model=joblib.load('./LR_model.sav')

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator StandardScaler from version 0.20.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator Pipeline from version 0.20.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [17]:
LR_result=logit_model.predict(get_matRx_4_gvn_Year1)

In [18]:
print(LR_result)

[1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 1 0 0]


In [19]:
y_predicted_proba = logit_model.predict_proba(get_matRx_4_gvn_Year1)

In [20]:
print(y_predicted_proba)

[[2.15451550e-01 7.84548450e-01]
 [9.99925216e-01 7.47844362e-05]
 [9.99531075e-01 4.68925182e-04]
 [7.28836737e-01 2.71163263e-01]
 [9.08382290e-01 9.16177096e-02]
 [9.89748500e-01 1.02515002e-02]
 [9.77331371e-01 2.26686294e-02]
 [9.99993768e-01 6.23249975e-06]
 [8.51271819e-01 1.48728181e-01]
 [9.76668808e-01 2.33311925e-02]
 [9.86345612e-01 1.36543885e-02]
 [3.87396415e-02 9.61260358e-01]
 [4.66798826e-01 5.33201174e-01]
 [9.53351466e-01 4.66485344e-02]
 [7.00143395e-01 2.99856605e-01]
 [9.46852550e-01 5.31474501e-02]
 [9.99651525e-01 3.48475014e-04]
 [9.21179028e-01 7.88209715e-02]
 [6.34170206e-03 9.93658298e-01]
 [8.89306817e-01 1.10693183e-01]
 [4.48514010e-01 5.51485990e-01]
 [9.45049341e-01 5.49506585e-02]
 [9.94316346e-01 5.68365401e-03]
 [4.60050966e-01 5.39949034e-01]
 [8.39390771e-01 1.60609229e-01]
 [9.99535479e-01 4.64521433e-04]
 [7.31231969e-02 9.26876803e-01]
 [7.65831745e-01 2.34168255e-01]
 [9.92673209e-01 7.32679087e-03]
 [1.57038684e-01 8.42961316e-01]
 [9.988200

In [21]:
win_loss_prdctd_data = pd.DataFrame(columns=['Team','0', '1', '2', '3','4', '5', '6','7', '8', '9','10', '11', '12', '13','14', '15', '16','17', '18', '19','20', '21', '22', '23','24', '25', '26','27', '28', '29', '30', '31'])
win_loss_prdctd_data["Team"] = ['0', '1', '2', '3','4', '5', '6','7', '8', '9','10', '11', '12', '13','14', '15', '16','17', '18', '19','20', '21', '22', '23','24', '25', '26','27', '28', '29', '30', '31']
win_loss_prdctd_data.head()

,Team,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
def prediction_4_season(season):
    a = season
    win_loss_prdctd_data = pd.DataFrame(columns=['Team','0', '1', '2', '3','4', '5', '6','7', '8', '9','10', '11', '12', '13','14', '15', '16','17', '18', '19','20', '21', '22', '23','24', '25', '26','27', '28', '29', '30', '31'])
    win_loss_prdctd_data["Team"] = ['0', '1', '2', '3','4', '5', '6','7', '8', '9','10', '11', '12', '13','14', '15', '16','17', '18', '19','20', '21', '22', '23','24', '25', '26','27', '28', '29', '30', '31']
#     switcher = { 
#         2002: 0, 2003: 1, 2004: 2, 2005: 3, 2006: 4, 2007: 5, 2008: 6, 2009: 7, 2010: 8, 2011: 9, 2012: 10, 2013: 11, 2014: 12, 2015: 13, 2016: 14, 2017: 15, 2018: 16
#     }
#     season = switcher.get(season, "nothing")
   
    print("Predicting for the Season ")
    print(a)
#     getting the result of prediction by passing the season specific data to the model prepared.

    logit_model=joblib.load('./LR_model.sav')
    LR_result=logit_model.predict_proba(get_matRx_4_gvn_Year(season))
    
    for i, row in win_loss_prdctd_data.iterrows():
        for j in range(1,33):
            if(row['Team']!=str(j-1)):
                x_Prob = LR_result[i]
                y_Prob = LR_result[j-1]
                prob_diff = abs(x_Prob[1] - y_Prob[1])
                row[j] = 1 if (x_Prob[1] > y_Prob[1]) else 0
            else:
                row[j] = "Same Team"
                    
    return win_loss_prdctd_data
    

In [23]:
prediction_Result = prediction_4_season(2012)
prediction_Result

Predicting for the Season 
2012


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator StandardScaler from version 0.20.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.20.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator Pipeline from version 0.20.3 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\preprocessing\data.py:323: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by MinMaxScaler.
  return self.partial_fit(X, y)


,Team,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,0,Same Team,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,1,1
1,1,0,Same Team,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,2,0,1,Same Team,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,3,0,1,1,Same Team,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,1,0,1,1,0,1,1,0,1,1,0,1,1
4,4,0,1,1,0,Same Team,1,1,1,0,1,1,0,0,1,0,1,1,1,0,0,0,1,1,0,0,1,0,0,1,0,1,1
5,5,0,1,1,0,0,Same Team,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1
6,6,0,1,1,0,0,1,Same Team,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1
7,7,0,0,0,0,0,0,0,Same Team,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,8,0,1,1,0,1,1,1,1,Same Team,1,1,0,0,1,0,1,1,1,0,1,0,1,1,0,0,1,0,0,1,0,1,1
9,9,0,1,1,0,0,1,1,1,0,Same Team,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1


In [24]:
dataframe=pd.read_csv("NFL_datasetForML.csv")
team_Names = dataframe['team'].unique()
team_Names = team_Names.tolist()
team_Names = sorted(team_Names)
team_Names

['atl',
 'buf',
 'car',
 'chi',
 'cin',
 'cle',
 'clt',
 'crd',
 'dal',
 'den',
 'det',
 'gnb',
 'htx',
 'jax',
 'kan',
 'mia',
 'min',
 'nor',
 'nwe',
 'nyg',
 'nyj',
 'oti',
 'phi',
 'pit',
 'rai',
 'ram',
 'rav',
 'sdg',
 'sea',
 'sfo',
 'tam',
 'was']

In [25]:
# Renaming of the rows
prediction_Result["Team"] = team_Names
prediction_Result

,Team,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31
0,atl,Same Team,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,1,1
1,buf,0,Same Team,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,car,0,1,Same Team,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,chi,0,1,1,Same Team,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,1,0,1,1,0,1,1,0,1,1,0,1,1
4,cin,0,1,1,0,Same Team,1,1,1,0,1,1,0,0,1,0,1,1,1,0,0,0,1,1,0,0,1,0,0,1,0,1,1
5,cle,0,1,1,0,0,Same Team,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1
6,clt,0,1,1,0,0,1,Same Team,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1
7,crd,0,0,0,0,0,0,0,Same Team,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,dal,0,1,1,0,1,1,1,1,Same Team,1,1,0,0,1,0,1,1,1,0,1,0,1,1,0,0,1,0,0,1,0,1,1
9,den,0,1,1,0,0,1,1,1,0,Same Team,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1


In [26]:
team_Names.insert(0,'Team')
team_Names

['Team',
 'atl',
 'buf',
 'car',
 'chi',
 'cin',
 'cle',
 'clt',
 'crd',
 'dal',
 'den',
 'det',
 'gnb',
 'htx',
 'jax',
 'kan',
 'mia',
 'min',
 'nor',
 'nwe',
 'nyg',
 'nyj',
 'oti',
 'phi',
 'pit',
 'rai',
 'ram',
 'rav',
 'sdg',
 'sea',
 'sfo',
 'tam',
 'was']

In [27]:
prediction_Result.columns = team_Names
prediction_Result.set_index('Team')
prediction_Result

,Team,atl,buf,car,chi,cin,cle,clt,crd,dal,den,det,gnb,htx,jax,kan,mia,min,nor,nwe,nyg,nyj,oti,phi,pit,rai,ram,rav,sdg,sea,sfo,tam,was
0,atl,Same Team,1,1,1,1,1,1,1,1,1,1,0,1,1,1,1,1,1,0,1,1,1,1,1,1,1,0,1,1,0,1,1
1,buf,0,Same Team,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,car,0,1,Same Team,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
3,chi,0,1,1,Same Team,1,1,1,1,1,1,1,0,0,1,0,1,1,1,0,1,0,1,1,0,1,1,0,1,1,0,1,1
4,cin,0,1,1,0,Same Team,1,1,1,0,1,1,0,0,1,0,1,1,1,0,0,0,1,1,0,0,1,0,0,1,0,1,1
5,cle,0,1,1,0,0,Same Team,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1
6,clt,0,1,1,0,0,1,Same Team,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1
7,crd,0,0,0,0,0,0,0,Same Team,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
8,dal,0,1,1,0,1,1,1,1,Same Team,1,1,0,0,1,0,1,1,1,0,1,0,1,1,0,0,1,0,0,1,0,1,1
9,den,0,1,1,0,0,1,1,1,0,Same Team,1,0,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,1,1
